# MLP on MINST Dataset

In [1]:
Pkg.update()
Pkg.add("MNIST")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Nothing to be done


In [7]:
using MNIST
features = trainfeatures(1)
label = trainlabel(1)

trainX, trainY = traindata()
testX, testY = testdata()

(
[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0],

[7.0,2.0,1.0,0.0,4.0,1.0,4.0,9.0,5.0,9.0  …  7.0,8.0,9.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0])

In [1]:
abstract Layer
abstract Nonlinearity <: Layer
abstract LossCriteria <: Layer

In [2]:
# Define the Fully Connected layers
type FCLayer <: Layer
    W           :: Array{Float64}
    last_input  :: Array{Float64}
    last_output :: Array{Float64}
    last_loss   :: Array{Float64}

    function FCLayer(i, o)
        return new(rand(o,i), zeros(i), zeros(o), zeros(o))
    end
end

function forward(l::FCLayer, x::Array{Float64,2})
    @assert size(x) == (size(l.W)[2],1)
    l.last_input  = x
    l.last_output = l.W * x # matrix multiplication
    l.last_output
end

function backward(l::FCLayer, loss::Array{Float64,1})
    @assert size(loss) == (size(l.W)[1],1)
    l.last_loss = loss
    l.W'*loss 
end

function gradient(l::FCLayer, loss::Array{Float64,1})
    @assert size(loss) == (size(l.W)[1],1)
    l.last_loss = loss
    l.W .* loss
end
FCLayer(10,20)


FCLayer([0.454393 0.762753 … 0.342354 0.697576; 0.431816 0.796653 … 0.756396 0.63993; … ; 0.128833 0.101582 … 0.408479 0.951413; 0.216083 0.999274 … 0.306752 0.483132],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [3]:
# Define the ReLu layers
type ReLu <: Nonlinearity
    alpha       :: Float64
    last_input  :: Array{Float64}
    last_output :: Array{Float64}
    last_loss   :: Array{Float64}
    function ReLu(alpha::Float64 = 1.0)
        @assert alpha >= 0.
        return new(alpha, Float64[], Float64[], Float64[])
    end
end

function forward(l::ReLu, x::Array{Float64})
    l.last_input  = x
    l.last_output = map(y -> max(0., y*l.alpha), x)
    l.last_output
end

function backward(l::ReLu, loss::Array{Float64})
    @assert size(l.last_input) == size(loss)
    l.last_loss = loss
    map(idx -> l.last_input[idx]>=0 ? l.last_input[idx]*l.alpha*loss[idx] : 0., indices(x))
end

ReLu()

ReLu(1.0,Float64[],Float64[])

In [4]:
type CrossEntropyLoss <: LossCriteria
    last_loss  :: Array{Float64}
    last_input :: Array{Float64}
end    

function forward(l::CrossEntropyLoss, y::Array{Float64}, label::Array{Int32})
    """
    [label]  label[i] == 1 iff the data is classified to class i
    [y]      final input to the loss layer
    """
    @assert min(label) >= 0 && sum(label) == 1
    @assert size(y) == size(label)
    sum(-log(e .^ y ./ sum(e .^ y)) .* label)
end

function backward(l::CrossEntropyLoss, x::Array{Float64}, label::Array{Int32})
    """
    [label]  label[i] == 1 iff the data is classified to class i
    [y]      final input to the loss layer
    """
    @assert min(label) >= 0 && sum(label) == 1
    @assert size(x) == size(loss)
    y = e.^x / sum(e.^x)
    c = filter(i -> label[i] == 1, indices(label))[1]
    map(j -> c==j ? y[c]*(1-y[c]) : -y[c]*y[j], indices(x))
end
CrossEntropyLoss()

CrossEntropyLoss()

In [5]:
abstract NN
type SequentialNet <: NN
    layers :: Array{Layer}
    lossfn :: LossCriteria
    function SequentialNet(layers::Array{Layer}, lossfn::LossCriteria)
        return new(layers, lossfn, Array{Float64}[], Array{Float64}[])
    end
end

function forward(net::SequentialNet, x::Array{Float64}, label::Array)
    local inp = x
    for i = 1:length(NN)
        inp = forward(net.layers[i], inp)
    end
    forward(net.lossfn, inp, label)
end

function backward(net::SequentialNet, label::Array)
    @assert size(dldy) == size(net.inputs[end])
    dldy = backward(net.lossfn, net.inputs[end], label)
    for i = length(net.layers):-1:1
        net.losses[i] = backward(net.layers[i], dldy)
    end
    dldy
end

backward (generic function with 4 methods)

In [6]:
layers = [
    FCLayer(784, 196),
    ReLu(),
    FCLayer(196, 49),
    ReLu(),
    FCLayer(49, 10)
]
criteria = CrossEntropyLoss()
net = SequentialNet(layers, criteria)

function sgd(net::SequentialNet, batch_X, batch_Y)
    local gradients 
    for b = 1:length(batch_X)
        
    end
end

function trainer(net::SequentialNet, X, Y)
    for epo = 1:100
        
    end
end

SequentialNet(Layer[FCLayer([0.386414 0.190032 … 0.242685 0.218318; 0.887087 0.351738 … 0.733746 0.762715; … ; 0.51793 0.394179 … 0.733619 0.129296; 0.566743 0.291666 … 0.340952 0.27762],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),ReLu(1.0,Float64[],Float64[]),FCLayer([0.940246 0.343635 … 0.142609 0.459007; 0.599952 0.583976 … 0.503514 0.145608; … ; 0.917356 0.247246 … 0.41323 0.660878; 0.782501 0.234676 … 0.87153 0.428659],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),ReLu(1.0,Float64[],Float64[]),FCLayer([0.424201 0.0713058 … 0.671137 0.00431872; 0.66011 0.355768 

In [11]:
trainX = trainX'
testX = testX'
size(trainX'), size(testX)

((60000,784),(10000,784))

In [12]:
size(trainY)

(60000,)